# Regresión Lineal Ordinal

### Xukai Chen & Julio 
### Curso: TDM - 2022/2023



Para  poder entender el concepto esta regresión y el problema a resolver, debemos recordar y recurrir a los conceptos basicos del data science, las variables.

Como ya sabemos, tanto regresión como clasificación, son un modelo de estimacion que tienen como objetivo predecir una relacion de dependencia entre una variable independiente  *Y* y una o varias variables dependientes  *X*.

<img src="./images/regr_cfc.png" alt="regre_cfc.png" width="600"/>

#### Los ejemplos más destacados de ambos tipos de algoritmos 
* Regresión lineal: la variable independiente a predecir es del tipo **continuo**
  * El peso y la altura de una persona
  * salario de una persona
  * distancia de un recorrido 
  * ...
  
* Regresion logística: mientras este predice variables **nominales**
  * Frutas y verduras
  * region
  * grupo sanguineo
  * ...

Es decir tanto el algoritmo de regresión como el de clasificación surgieron para adecuar a las caracteristas de estos tipos de variable, para poder predecirlos con alta precision y exhaustividad.

Sin embargo, existe todavía un tipo de variable que es muy comun, se trata de una variable **ordinal**, por ejemplo, el nivel de satifaccion de las encuestas, la nota descriptica de la evaluacion del trimestre, puestos conseguidos en los deportes competitivos...

Como se puede apreciar es como una mezcla de los dos tipos de variables mencionados anteriormente, donde no puede tomar cualquier valor entre dos valores consecutivos pero mantiene cierta relacion de continuidad, el motivo por el cual tampoco puede considerarse como variable nominal, ya que los valores no son independientes entre ellos, es decir, es erronea la prediccion de una variable ordinal si tuviera que elegir entre *Muy satisfecho* y *Muy insatisfecho*.

<img src="./images/satisfaccion.jpg" alt="satisfaccion.jpg" width="1000"/>

##### A continuación vamos a ver varios metodos regresion y clasificacion los cuales podemos obtener una aproximacion a la solucion:

## Inicialización

Inicilización de pyspark

In [ ]:
# primero lo descargamos en local
import pyspark                                 # only run after findspark.init()
from pyspark.sql import SparkSession
import urllib
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Funciones útiles 

A continuación se proporcionan algunas de las funciones que pueden ser útiles, por ejemplo, las funciones para la evaluación de los modelos creados.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import  ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark.sql.functions as f 
import numpy as np

########
# Creación del dataframe a partir de un ficheroE
########
def create_spark_dataframe(file):
    df = spark.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load(file)
    return df

def accuracy(predictions, label, prediction="prediction"):    
    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol = prediction, metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    print("Classification accuracy = %g" % (accuracy))
    return accuracy

def f1(predictions, label, prediction="prediction"):
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol=prediction, metricName="f1")
    f1 = evaluator.evaluate(predictions)
    print("Classification f1 score= " + str(f1))
    return f1

def calcular_matriz_confusion(predictions,label):      
    preds_and_labels = predictions.select(['prediction', label])
    # hay que convertir las columnas prediction label en una tupla
    metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
    cm = metrics.confusionMatrix()
    
    # para mostrar las etiquetas...muy lioso para poca cosa
    class_temp = predictions.select(label).groupBy(label)\
                            .count().sort('count', ascending=False).toPandas()
    class_names = class_temp[label].values.tolist()
    cm = cm.toArray().astype(np.int64)

    fig, ax = plt.subplots(figsize=(4,4))
    ConfusionMatrixDisplay(cm, display_labels=class_names).plot(ax =ax)


## Carga de datos

In [ ]:
file = "./carkids.csv"
# ahora llamamos a la función
df = create_spark_dataframe(file)



A continuación se describe cada una de las columnas del dataframe:

* `Sales`: Ventas unitarias de cierto producto (en miles) en cada ubicación
* `CompPrice`: Precio cobrado en cada ubicación
* `Income`:Nivel de ingresos 
* `Advertising`:  Presupuesto de publicidad local para la empresa en cada ubicación
* `Population`: población en la región
* `Price`: Costes
* `Age`: Edad media de la población
* `Education`: Nivel de educación en cada lugar
* `Urban`: Un factor con niveles ``No`` y ``Sí`` para indicar si la tienda
   está en una zona urbana o rural
* `ShelveLoc`: Un factor con niveles ``Malo``, ``Bueno`` y ``Medio`` que indica el
   calidad de la ubicación
* `US`: Un factor con niveles ``No`` y ``Sí`` para indicar si la tienda
   está en los Estados Unidos o no



In [ ]:
from pyspark.sql.functions import when
_df = df.withColumn('ShelveLocIndex', when(df.ShelveLoc == 'Bad', 0.0).when(df.ShelveLoc == 'Medium', 1.0).otherwise(2.0))
_df = _df.withColumn('volumen_ventas', when(df.Sales<5.5, 0.0).when(df.Sales<9, 1.0).otherwise(2.0))
_df.show(5)

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Urban", outputCol="UrbanIndex")
indexed_model = indexer.fit(_df)
_df = indexed_model.transform(_df)
indexer = StringIndexer(inputCol="US", outputCol="USIndex")
indexed_model = indexer.fit(_df)
_df = indexed_model.transform(_df)
_df = _df.drop('Urban').drop('US').drop('Sales').drop('ShelveLoc')
_df.show(5)

### Pregunta 6

Comenzamos con la preparación de los datos para la construcción del modelo. El objetivo es predecir el valor de la columna `volumen_ventas` a partir del resto de columnas.

Compararemos 2 modelos: Decision trees y  Naïve Bayes


Combinar en un `VectorAssembler` todas las columnas excepto la columna `volumen_ventas`, dando como salida una nueva columna `features`. Este transformador tomará el dataframe resultado del apartado anterior como entrada. El dataframe de salida se debe llamar `df_features`.


In [ ]:
# Sol:
from pyspark.ml.feature import VectorAssembler

cols = _df.columns
cols.remove('volumen_ventas')
assembler = VectorAssembler( inputCols = cols,
                             outputCol ='features')
df_features = assembler.transform(_df)
df_features.show(5)

### Pregunta 7. 

División entre entrenamiento y test.
* 70% de valores para entrenar 
* 30% para test. 


In [ ]:
# Sol:
df_features = df_features.select(['volumen_ventas', 'features'])
train, test = df_features.randomSplit([0.7, 0.3], seed = 1234)
df_features.show(5)

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

mlr = LogisticRegression(labelCol="volumen_ventas", family="multinomial")

paramGrid_mlr = ParamGridBuilder().baseOn([mlr.labelCol, 'volumen_ventas'])\
             .baseOn([mlr.family, 'multinomial'])  \
             .addGrid(mlr.regParam, [0.01, 0.1, 0.5, 1.0, 2.0]) \
            .addGrid(mlr.elasticNetParam, [0.0, 0.5, 1.0]) \
             .build()

cvEvaluator= MulticlassClassificationEvaluator(metricName="f1", labelCol='volumen_ventas')

cv_mlr = CrossValidator(estimator=mlr, estimatorParamMaps=paramGrid_mlr, evaluator=cvEvaluator)
cvModel_mlr = cv_mlr.fit(train)

def printBestParam(cvModel):
    bestModel = cvModel.bestModel

    bestParams = bestModel.extractParamMap()
    for key in bestParams.keys():
        print(key,': ', bestParams[key])

printBestParam(cvModel_mlr)

In [ ]:
mlr_best = LogisticRegression(labelCol="volumen_ventas", family="multinomial", regParam=0.01, elasticNetParam=0.1)
mlrModel_best = mlr_best.fit(train)

print("Coefficients: \n" + str(mlrModel_best.coefficientMatrix))
print("Intercept: " + str(mlrModel_best.interceptVector))

trainingSummary = mlrModel_best.summary

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

In [ ]:
predictions_mlr = mlrModel_best.transform(test)
f1(predictions_mlr, 'volumen_ventas')
calcular_matriz_confusion(predictions_mlr, 'volumen_ventas')

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lr = LogisticRegression(labelCol="volumen_ventas", family="binomial")

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(labelCol="volumen_ventas", classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(train)

# score the model on test data.
predictions_ovr = ovrModel.transform(test)

f1(predictions_ovr, "volumen_ventas")
calcular_matriz_confusion(predictions_mlr, 'volumen_ventas')


In [ ]:
df_features_l = df_features.withColumn('volumen_ventas_1', when(df_features.volumen_ventas == 0.0, 0.0).otherwise(1.0))
df_features_l = df_features_l.withColumn('volumen_ventas_2', when(df_features_l.volumen_ventas <= 1.0, 0.0).otherwise(1.0))
df_features_l = df_features_l.select(['volumen_ventas', 'volumen_ventas_1','volumen_ventas_2', 'features'])
train_l, test_l = df_features_l.randomSplit([0.7, 0.3], seed = 1234)
df_features_l.show(5)

In [ ]:
from pyspark.ml.regression import GeneralizedLinearRegression

glr = GeneralizedLinearRegression(labelCol="volumen_ventas_1")

paramGrid_glr = ParamGridBuilder().baseOn([glr.labelCol, 'volumen_ventas_1'])\
             .baseOn([glr.predictionCol, 'prediction_1'])\
             .baseOn([glr.family, 'binomial'])  \
             .addGrid(glr.link, ["logit", "probit"]) \
             .addGrid(glr.regParam, [0.01, 0.1, 0.5, 1.0, 2.0]) \
             .build()

cvEvaluator= MulticlassClassificationEvaluator(metricName="f1", labelCol='volumen_ventas_1', predictionCol="prediction_1")

cv_glr = CrossValidator(estimator=glr, estimatorParamMaps=paramGrid_glr, evaluator=cvEvaluator)
cvModel_glr = cv_glr.fit(train_l)

printBestParam(cvModel_glr)

glr_2 = GeneralizedLinearRegression(labelCol="volumen_ventas_2")

paramGrid_glr_2 = ParamGridBuilder().baseOn([glr_2.labelCol, 'volumen_ventas_2'])\
            .baseOn([glr_2.predictionCol, 'prediction_2'])\
             .baseOn([glr_2.family, 'binomial'])  \
             .addGrid(glr_2.link, ["logit", "probit"]) \
             .addGrid(glr_2.regParam, [0.01, 0.1, 0.5, 1.0, 2.0]) \
             .build()

cvEvaluator= MulticlassClassificationEvaluator(metricName="f1", labelCol='volumen_ventas_2', predictionCol="prediction_2")

cv_glr_2 = CrossValidator(estimator=glr_2, estimatorParamMaps=paramGrid_glr_2, evaluator=cvEvaluator)
cvModel_glr_2 = cv_glr_2.fit(train_l)

printBestParam(cvModel_glr_2)

In [ ]:
predictions_1 = cvModel_glr.transform(test_l)
test_ll = predictions_1.select(['volumen_ventas', 'volumen_ventas_1', 'features', 'prediction_1'])
predictions_2 = cvModel_glr_2.transform(test_ll)
predictions_2.show(5, False)
df_predictions = predictions_2.withColumn('prediction', when(predictions_2.prediction_1 < 0.45, 0.0).when(predictions_2.prediction_2 >= 0.5, 2.0).otherwise(1.0))
df_predictions.show(5, False)
f1(df_predictions, "volumen_ventas")
calcular_matriz_confusion(df_predictions, "volumen_ventas")

In [ ]:
lr_1 = LogisticRegression(labelCol="volumen_ventas_1", family="binomial", predictionCol="prediction_1")
lr_1_Model = lr_1.fit(train_l)
predictions_1 = lr_1_Model.transform(test_l)
lr_2 = LogisticRegression(labelCol="volumen_ventas_2", family="binomial", predictionCol="prediction_2")
lr_2_Model = lr_2.fit(train_l)
test_ll = predictions_1.select(['volumen_ventas', 'volumen_ventas_1', 'volumen_ventas_1', 'features', 'prediction_1'])
predictions_2 = lr_2_Model.transform(test_ll)
predictions_2.show(5, False)
df_predictions = predictions_2.withColumn('prediction', when(predictions_2.prediction_1 == 0.0, 0.0).when(predictions_2.prediction_2 == 1.0, 2.0).otherwise(1.0))
df_predictions.show(5, False)
f1(df_predictions, "volumen_ventas")
calcular_matriz_confusion(df_predictions, "volumen_ventas")